In [1]:
import pandas as pd
import xgboost as xgb
import mlflow
import mlflow.xgboost
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from mlflow.models import infer_signature

# 데이터 로딩
data = pd.read_csv('../data/dataset.csv')

# Tracking Server 설정
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000") # 환경 변수를 설정했지만, 확실하게 설정

In [2]:
# 타겟과 특성 분리
X = data.drop('Default', axis=1)
y = data['Default']

In [3]:
# 전처리를 위한 컬럼 구분
categorical_cols = ['State', 'BankState', 'NewExist', 'UrbanRural', 'RealEstate']
numerical_cols = ['DisbursementGross', 'GrAppv', 'daysterm']

In [4]:
# 전처리 파이프라인 구성 num은 수치형, cat은 범주형
preprocessor = ColumnTransformer([
    ('num', StandardScaler(), numerical_cols),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_cols)
])

In [5]:
# 전처리 실행
X_processed = preprocessor.fit_transform(X)

In [6]:
# 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(X_processed, y, test_size=0.2, random_state=42)

In [7]:
# XGBoost 모델 학습
model = xgb.XGBClassifier(eval_metric='logloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [9]:
# 2. MLflow를 사용하여 모델 로깅 및 등록
mlflow.set_experiment("assignment2")

with mlflow.start_run(run_name="XGBoost"):

    # 모델 파라미터 기록
    mlflow.log_params(model.get_params())

    # 성능 지표 기록 
    y_pred = model.predict(X_test)
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # 성능 평가 지표 계산
    acc = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    mlflow.log_metric("roc_auc", roc_auc)

    # 모델 로깅 및 Model Registry 등록
    signature = infer_signature(X_train, y_pred)
    mlflow.xgboost.log_model(model, "model", registered_model_name="LoanDefaultModel", signature=signature, input_example=X_test[:1])

    print(f"모델 등록 완료. ROC-AUC: {roc_auc:.4f}")

c:\Users\SSAFY\venvs\mlflow-env\Lib\site-packages\xgboost\sklearn.py:1028: UserWarning: [17:23:07] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\c_api\c_api.cc:1427: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  self.get_booster().save_model(fname)
c:\Users\SSAFY\venvs\mlflow-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Successfully registered model 'LoanDefaultModel'.
2025/10/10 17:23:13 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: LoanDefaultModel, version 1


모델 등록 완료. ROC-AUC: 0.9718
🏃 View run XGBoost at: http://127.0.0.1:5000/#/experiments/1/runs/973faaf29d0b4a269b954397ef42b986
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/1


Created version '1' of model 'LoanDefaultModel'.
